In [2]:
import pickle
import numpy as np
from classes import classes, colors
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
with open("lpf_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

with open("imputed_sensor_data.pkl", "rb") as f:
    sensor_data_raw = pickle.load(f)

with open("sensor_labels.pkl", "rb") as f:
    labels = pickle.load(f)

with open("interpolation_functions.pkl", "rb") as f:
    interp_funcs = pickle.load(f)

In [4]:
colors_reg = ["#DB073D", "#DBA507", "#0D6986"]
fig = make_subplots(rows=2, cols=1,
                    # shared_yaxes=True,
                    vertical_spacing=0.2,
                    subplot_titles=("LPF Data",
                                    "Interpolated Data"))
matrix = 0
sensor = 0
num_samples = 3
time_offset = 20
time_offset_end = time_offset + num_samples
starts = []
ends = []
for heater_idx in range(3):
    heater_data = sensor_data[f"mat_{matrix}"][sensor][heater_idx]
    time_arr = heater_data["Time Since PowerOn"].values[time_offset:time_offset_end]
    starts.append(time_arr[0])
    ends.append(time_arr[-1])
    fig.add_trace(go.Scatter(x=time_arr,
                             y=heater_data["Filtered"].values[time_offset:time_offset_end],
                             mode="lines+markers",
                             marker_color=colors_reg[heater_idx],
                             name=f"H.S. {heater_idx}",
                             showlegend=False,
                             legendgroup=f"H.S. {heater_idx}"),
                  row=1, col=1)

start = min(starts)
end = min(ends)
num_samples = 6
time_arr = np.linspace(start, end, num_samples)
for heater_idx in range(3):
    heater_data = sensor_data[f"mat_{matrix}"][sensor][heater_idx]
    interp_resistance = interp_funcs[f"mat_{
        matrix}"][sensor][heater_idx](time_arr)
    fig.add_trace(go.Scatter(x=time_arr,
                             y=interp_resistance,
                             mode="lines+markers",
                             marker_color=colors_reg[heater_idx],
                             name=f"H.S. {heater_idx}",
                             showlegend=False,
                             legendgroup=f"H.S. {heater_idx}"),
                  row=2, col=1)

fig.update_yaxes(title_text="Resistance (Ohms)", row=1,
                 col=1, tickformat=".0e")
fig.update_yaxes(title_text="Resistance (Ohms)", row=2,
                 col=1, tickformat=".0e")

fig.update_xaxes(title_text="Time since power on (ms)",
                 row=1, col=1, tickformat=".0e")
fig.update_xaxes(title_text="Time since power on (ms)",
                 row=2, col=1, tickformat=".0e")

fig.update_layout(height=500, width=625,
                  title_x=0.5,
                  font_family="Times New Roman",
                  font_color="black",
                  title_font_family="Times New Roman",
                  title=dict(text="Effects of Interpolation"),
                  margin=dict(t=50, r=10, b=0, l=0))
fig.write_image(f"figures/interp.svg")
fig.show()

In [7]:
def plot_sensor_it(matrix, sensor_idx, sensor_data, labels, classes, colors,
                   save_pdf=False,
                   include_labels=False,
                   skip=0,
                   end_skip=0):
    fig = go.Figure()

    sensors = sensor_data[f"mat_{matrix}"]
    for profile_idx in sensors[sensor_idx]:
        fig.add_trace(go.Scatter(x=sensors[sensor_idx][profile_idx]["Time Since PowerOn"].values[skip:-end_skip],
                                 y=sensors[sensor_idx][profile_idx]["Resistance Gassensor"].values[skip:-end_skip],
                                 mode="lines",
                                 showlegend=False,
                                 name=f"H.S. {profile_idx}"))

    if include_labels:
        for label in labels[f"mat_{matrix}"]:
            fig.add_vrect(x0=label["start"],
                          x1=label["end"],
                          annotation_text=classes[label["label"]],
                          annotation_position="top left",
                          fillcolor=colors[label["label"]],
                          opacity=0.4,
                          line_width=0)

    fig.update_yaxes(title_text="Resistance (Ohms)",
                     type="log", tickformat=".0e")
    fig.update_xaxes(title_text="Time since power on (ms)", tickformat=".0e")

    title = f"Matrix {matrix}, Sensor {sensor_idx}"
    fig.update_layout(height=300, width=625,
                      title_x=0.5,
                      font_family="Times New Roman",
                      font_color="black",
                      title_font_family="Times New Roman",
                      title=dict(text=title, pad=dict(t=0, r=0, b=0, l=0)),
                      margin=dict(t=50, r=10, b=0, l=0))

    if save_pdf:
        f_name = f"figures/{matrix}_{sensor_idx}"
        if include_labels:
            f_name += "_w_labels"
        f_name += ".svg"
        fig.write_image(f_name)
    fig.show()

In [12]:
plot_sensor_it(1, 6,
               sensor_data, labels,
               classes, colors,
               save_pdf=False,
               include_labels=True,
               skip=2400,
               end_skip=1)

In [19]:
with open("imputed_sensor_data.pkl", "rb") as f:
    sensor_data = pickle.load(f)

In [30]:
plot_sensor_it(0, 7,
               sensor_data_raw, labels,
               classes, colors,
               save_pdf=True,
               include_labels=False,
               skip=2400,
               end_skip=1)

In [4]:
hp_colors = ["#9e062c", "#789e06", "#069e78", "#2c069e"]
HP_323 = {
    "x": [6.02, 6.3, 6.58, 6.86, 9.8, 12.74, 13.02, 14.98, 16.94, 18.9],
    "y": [70, 350, 350, 210, 210, 210, 350, 350, 350, 350],
    "name": "HP-323",
    "lines": {
        "x": [(0, 0), (0, 6.02), (6.02, 6.02), (6.02, 6.58), (6.58, 6.58),
              (6.58, 12.74), (12.74, 12.74), (12.74, 18.9), (18.9, 18.9)],
        "y": [(0, 70), (70, 70), (70, 350), (350, 350), (350, 210),
              (210, 210), (210, 350), (350, 350), (350, 0)]
    }
}

HP_354 = {
    "x": [0.7, 0.98, 2.38, 6.58, 7.28, 7.98, 8.68, 9.38, 10.08, 10.78],
    "y": [320, 100, 100, 100, 200, 200, 200, 320, 320, 320],
    "name": "HP-354",
    "lines": {
        "x": [(0, 0), (0, 0.7), (0.7, 0.7), (0.7, 6.58), (6.58, 6.58),
              (6.58, 8.68), (8.68, 8.68), (8.68, 10.78), (10.78, 10.78)],
        "y": [(0, 320), (320, 320), (320, 100), (100, 100), (100, 200),
              (200, 200), (200, 320), (320, 320), (320, 0)]
    }
}

HP_411 = {
    "x": [6.02, 6.3, 12.32, 12.6, 12.88, 15.68, 18.62, 18.9, 21.7, 24.64],
    "y": [100, 320, 170, 320, 240, 240, 240, 320, 320, 320],
    "name": "HP-411",
    "lines": {
        "x": [(0, 0), (0, 6.02), (6.02, 6.02), (6.02, 6.3), (6.3, 6.3),
              (6.3, 12.32), (12.32, 12.32), (12.32, 12.6), (12.6, 12.6),
              (12.6, 18.62), (18.62, 18.62), (18.62, 24.64), (24.64, 24.64)],
        "y": [(0, 100), (100, 100), (100, 320), (320, 320), (320, 170),
              (170, 170), (170, 320), (320, 320), (320, 240),
              (240, 240), (240, 320), (320, 320), (320, 0)]
    }
}

HP_503 = {
    "x": [3.36, 3.64, 6.72, 7, 10.08, 13.44, 16.8, 20.16, 23.52, 26.88],
    "y": [210, 280, 280, 350, 350, 280, 210, 140, 70, 140],
    "name": "HP-503",
    "lines": {
        "x": [(0, 0), (0, 3.36), (3.36, 3.36), (3.36, 6.72), (6.72, 6.72),
              (6.72, 10.08), (10.08, 10.08), (10.08, 13.44), (13.44, 13.44),
              (13.44, 16.8), (16.8, 16.8), (16.8, 20.16), (20.16, 20.16),
              (20.16, 23.52), (23.52, 23.52), (23.52, 26.88), (26.88, 26.88)],
        "y": [(0, 210), (210, 210), (210, 280), (280, 280), (280, 350),
              (350, 350), (350, 280), (280, 280), (280, 210),
              (210, 210), (210, 140), (140, 140), (140, 70),
              (70, 70), (70, 140), (140, 140), (140, 0)]
    }
}

HP_332 = {
    "x": [],
    "y": [],
    "name": "HP-332",
    "lines": {
        "x": [],
        "y": []
    }
}

HP_414 = {
    "x": [],
    "y": [],
    "name": "HP-414",
    "lines": {
        "x": [],
        "y": []
    }
}

HP_504 = {
    "x": [],
    "y": [],
    "name": "HP-504",
    "lines": {
        "x": [],
        "y": []
    }
}

In [21]:
def plot_hp(hp_arr, colors, title: str, save_img=False):
    fig = make_subplots(rows=2, cols=2,
                        vertical_spacing=0.1,
                        horizontal_spacing=0.075,
                        subplot_titles=[x["name"] for x in hp_arr])

    for i, hp in enumerate(hp_arr):
        fig.add_trace(go.Scatter(x=hp["x"],
                                 y=hp["y"],
                                 mode="markers",
                                 name=hp["name"],
                                 legendgroup=hp["name"],
                                 showlegend=False,
                                 marker_color=colors[i]),
                      row=1 if i < 2 else 2,
                      col=(i % 2) + 1)

        fig.add_trace(go.Scatter(
            x=list(sum(hp["lines"]["x"], ())),
            y=list(sum(hp["lines"]["y"], ())),
            mode="lines",
            fill="toself",
            line=dict(
                color=colors[i],
                width=0
            ),
            fillcolor=colors[i],
            opacity=0.5,
            legendgroup=hp["name"],
            showlegend=False
        ),
            row=1 if i < 2 else 2,
            col=(i % 2) + 1)

    fig.update_layout(height=400, width=625,
                      title_x=0.5,
                      font_color="black",
                      font_family="Times New Roman",
                      title_font_family="Times New Roman",
                      title_font_size=16,
                      title=dict(text=title, pad=dict(t=0, r=0, b=0, l=0)),
                      margin=dict(t=40, r=10, b=0, l=0)
                      )
    fig.update_yaxes(ticksuffix="°C", range=[0, 360])
    fig.update_xaxes(ticksuffix="s")
    fig.update_annotations(font_size=14)

    fig.show()
    if save_img:
        f_name = f"figures/{title.lower().replace(" ", "_")}"
        f_name += ".svg"
        fig.write_image(f_name)

In [22]:
plot_hp([HP_354, HP_323, HP_411, HP_503],
        hp_colors, "Matrix 0 Heater Profiles", save_img=True)